# CareerVilage EDA

- 목표: 올바른 자원 봉사자에게 올바른 질문을 보낼 수 있어야합니다. 
- 가장 관련있는 전문가에게 관련 질문을 추천하는 방법을 개발하기

## Idea 1: Questions-Answers 매칭하기
- Idea 1 : questions_title에 있는 명사만 tokenizing으로 뽑아내서 professional_industry와 연결짓기
- Idea 2 : questions_body에서 NER(개체명인식)을 통해 중요한 개체를 찾아서 TF-idf 돌리기 => 유사한 질문목록 

## Idea 2: Answers의 글에서 명사/동사 Hashtags를 뽑아서 전문가별 Hashtags 만들기
- 이후 Tags와 연계하여 새로운 Questions와 매칭
- 전문가의 위치(location)/산업(industry)에 따라 질문의 해시태그로 1차 분류

## Idea 3: Tags 상위를 띄워서 홈페이지에 올리기
- Idea: 태그들 top10으로 띄워서 웹페이지에서 해당 태그 누르면 자동으로 해당 top 질문-답변 볼 수 있게 하는 것은 어떨까?

## Idea 4: Groups의 특징을 파악해서 Questions의 Token과 상관관계 뽑기

## Idea 5: Professional과 Student의 상관관계 확인
- 오프라인 매칭(발전방향)
- 향후에 professional이랑 같은 위치인 사람들끼리 오프라인 매칭 시켜주는 것도 좋을 듯?
- data_join에 따라 태그가 어떻게 변화하는지 확인해보는 것도 좋을 듯

### 1. Data Load

In [4]:
import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
answers = pd.read_csv('../Desktop/data-science-for-good-careervillage/answers.csv')
comments = pd.read_csv('../Desktop/data-science-for-good-careervillage/comments.csv')
emails = pd.read_csv("../Desktop/data-science-for-good-careervillage/emails.csv")
group_memberships = pd.read_csv('../Desktop/data-science-for-good-careervillage/group_memberships.csv')
groups = pd.read_csv('../Desktop/data-science-for-good-careervillage/groups.csv')
matches = pd.read_csv('../Desktop/data-science-for-good-careervillage/matches.csv')
professionals = pd.read_csv("../Desktop/data-science-for-good-careervillage/professionals.csv")
questions = pd.read_csv('../Desktop/data-science-for-good-careervillage/questions.csv')
school_memberships = pd.read_csv('../Desktop/data-science-for-good-careervillage/school_memberships.csv')
students = pd.read_csv('../Desktop/data-science-for-good-careervillage/students.csv')
tag_questions = pd.read_csv("../Desktop/data-science-for-good-careervillage/tag_questions.csv")
tag_users = pd.read_csv('../Desktop/data-science-for-good-careervillage/tag_users.csv')
tags = pd.read_csv('../Desktop/data-science-for-good-careervillage/tags.csv')

## 2. EDA

### 2-1. Answers

In [37]:
answers.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08 UTC+0000,<p>Hi. I joined the Army after I attended coll...
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02 UTC+0000,"<p>Dear Priyanka,</p><p>Greetings! I have answ..."
3,1a6b3749d391486c9e371fbd1e605014,7e72a630c303442ba92ff00e8ea451df,4ec31632938a40b98909416bdd0decff,2017-05-10 19:00:47 UTC+0000,<p>I work for a global company who values high...
4,5229c514000446d582050f89ebd4e184,17802d94699140b0a0d2995f30c034c6,2f6a9a99d9b24e5baa50d40d0ba50a75,2017-10-13 22:07:33 UTC+0000,I agree with Denise. Every single job I've had...


In [39]:
answers.answers_body[1]

"<p>Hi. I joined the Army after I attended college and received a Bachelor's Degree in Criminal Justice.  Commissioned officers enter the Military with a four year degree or receive officer training after joining and complete a tour. You can prepare yourself by taking Reserve Officer Training Corps (ROTC) while in high school or a university. You can also attend Officer Candidate School (OCS) after graduating from college or become commissioned by earning a professional degree. If you decide to earn a degree, think about what you would like to do while in the military.  The experience in the military was very rewarding. I wish you well and much success in your future. </p>"

In [11]:
answers.describe()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body
count,51123,51123,51123,51123,51122
unique,51123,10169,23110,51062,50401
top,ab0e04a3d71e4e6f8ef0b97a1eb8d26a,36ff3b3666df400f956f8335cf53e09e,8eb6ba7af57846acbfec5633e537192a,2016-10-13 00:52:24 UTC+0000,"<p>&lt;span style=""background-color: transpare..."
freq,1,1710,58,2,103


- 답변 갯수는 51122개
- answers_author_id의 unique값이 1/5로 줄어드는 것을 보아, 전문가가 여러번 답변함

- 답변한 사람들의 텍스트로 저자의 키워드를 해시태그로 뽑아내야 할 것 같음. 
##### 이후 해당 해시태그가 새로운 질문에 들어갈 경우, 해당 해시태그를 많이 답변해준 전문가에게 연결해야할 듯 보임.


### 2-2. Comments

In [12]:
comments.head()

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
0,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40 UTC+0000,"First, you speak to recruiters. They are train..."
1,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47 UTC+0000,Most large universities offer study abroad pro...
2,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04 UTC+0000,"First, I want to put you at ease that the oppo..."
3,73a6223948714c5da6231937157e4cb7,d02f6d9faac24997a7003a59e5f34bd3,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 17:53:28 UTC+0000,Your question submission was great! I just wan...
4,55a89a9061d44dd19569c45f90a22779,e78f75c543e84e1c94da1801d8560f65,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 14:51:53 UTC+0000,Thank you. I'm new to this site. I'm sorry if ...


In [40]:
comments.comments_body[1]

"Most large universities offer study abroad programs.  The study abroad programs are found on the schools website.  You may have to click into the Undergraduate or Graduate links.   I also recommend reaching out to the university counselors.  The have wealth of information and details that may not be on the website. \r\n\r\nMy son will be studying in France and Sweden next year through DePaul University.   The University of Southern California has a very strong international program in which most students participate.  It's a great opportunity to learn the culture of other countries while in school."

In [13]:
comments.describe()

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
count,14966,14966,14966,14966,14962
unique,14966,4231,9923,14905,11963
top,09fad9fdb89b4b7493e9328513ee279c,36ff3b3666df400f956f8335cf53e09e,4c6ae4bd2136472899c15b798eb9d3df,2016-07-22 22:13:05 UTC+0000,Thank you!
freq,1,317,30,3,388


- 코멘트 아이디 통해서 코멘트 많이 다는 사람들을 알 수 있지 않을까

### 2-3. Emails(보류)

In [14]:
emails.head()

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily
1,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily
2,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily
3,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily
4,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily


In [15]:
emails.describe() 

,emails_id
count,1.850101e+06
mean,1.362206e+06
std,6.449917e+05
min,1.060000e+02
25%,8.813900e+05
50%,1.400010e+06
75%,1.911846e+06
max,2.409829e+06


### 2-4. Group_memberships

In [16]:
group_memberships.head()

,group_memberships_group_id,group_memberships_user_id
0,eabbdf4029734c848a9da20779637d03,9a5aead62c344207b2624dba90985dc5
1,eabbdf4029734c848a9da20779637d03,ea7122da1c7b4244a2184a4f9f944053
2,eabbdf4029734c848a9da20779637d03,cba603f34acb4a40b3ccb53fe6681b5d
3,eabbdf4029734c848a9da20779637d03,fa9a126e63714641ae0145557a390cab
4,eabbdf4029734c848a9da20779637d03,299da113c5d1420ab525106c242c9429


In [17]:
group_memberships.describe()

,group_memberships_group_id,group_memberships_user_id
count,1038,1038
unique,46,727
top,f16d593910bf443598839021bbe97128,b0f6c44506444fb99e910dcc5836b5d8
freq,117,14


### 2-5. Groups

In [18]:
groups.head()

,groups_id,groups_group_type
0,eabbdf4029734c848a9da20779637d03,youth program
1,7080bf8dcf78463bb03e6863887fd715,youth program
2,bc6fc50a2b444efc8ec47111b290ffb8,youth program
3,37f002e8d5e442ca8e36e972eaa55882,youth program
4,52419ff84d4b47bebd0b0a6c1263c296,youth program


In [43]:
groups.groups_group_type.unique() 

array(['youth program', 'cause', 'professional network', 'club',
       'competition', 'mentorship program', 'interest group'],
      dtype=object)

In [19]:
groups.describe()

,groups_id,groups_group_type
count,49,49
unique,49,7
top,2db32f76e5524d57ace1012aa9fe1125,youth program
freq,1,33


##### 특정 프로그램을 통해 group_type이 결정되는 듯. 그룹끼리의 연관성이 있는지도 파악해볼 것

### 2-6. Matches

In [20]:
matches.head()

,matches_email_id,matches_question_id
0,1721939,332a511f1569444485cf7a7a556a5e54
1,1665388,332a511f1569444485cf7a7a556a5e54
2,1636634,332a511f1569444485cf7a7a556a5e54
3,1635498,332a511f1569444485cf7a7a556a5e54
4,1620298,332a511f1569444485cf7a7a556a5e54


In [21]:
matches.describe()

,matches_email_id
count,4.316275e+06
mean,1.478192e+06
std,5.895280e+05
min,1.060000e+02
25%,1.011924e+06
50%,1.544656e+06
75%,1.970642e+06
max,2.409829e+06


### 2-7. Professionals

In [22]:
professionals.head()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,2011-10-05 20:35:19 UTC+0000
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,2011-10-05 20:49:21 UTC+0000
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",NaN,NaN,2011-11-09 20:39:29 UTC+0000
4,e2d57e5041a44f489288397c9904c2b2,NaN,NaN,NaN,2011-12-10 22:14:44 UTC+0000


In [51]:
print('location:', professionals.professionals_location.unique())

print('Industry:', professionals.professionals_industry.unique())

location: [nan 'New York, New York' 'Boston, Massachusetts' ...
 'Kingston, Washington' 'Berwyn, Illinois' 'Woodland, California']
Industry: [nan 'Special Events' 'education, tech, media' ... 'Dev Sec Ops'
 'Electrical engineering Computer Software' 'Mobility and Entertainment']


In [23]:
professionals.describe()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
count,28152,25054,25576,26085,28152
unique,28152,2582,2470,22272,28000
top,668311ec0dbb4c1db6d5d8fb4e3d7e7e,"New York, New York",Telecommunications,--,2011-12-26 05:00:00 UTC+0000
freq,1,1337,3166,325,45


- 자 우리의 전문가들! 
###### 전문가의 위치(location)/산업(industry)에 따라 질문의 해시태그로 1차 분류를 할 수 있을 것
- 문제는 location과 industry가 너무 많음(각각 2000개)
- location: 미국인데 일단 52개 주별로 한번 나누기(현재는 도시별로 되어있는듯)
- Industry: 따라서 대분류 카테고리로 분류하는 것이 필요해보임(제조업/통신업/서비스업 등등으로..? 산업분류표 확인)
- 아 너무 많으니까 아예 페이지에서 분류카테고리를 만드는게 필요해보임. 너무 복잡쓰

### 2-8. Questions(핵심)

In [24]:
questions.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [25]:
questions.describe()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
count,23931,23931,23931,23931,23931
unique,23931,12329,23869,23739,23681
top,f74840697f3644009d22950d9165cafe,2fe767de78fa4dfd83f0021cf7712064,2016-05-17 22:14:06 UTC+0000,How do I become an IAS officer?,Accounting student at Towson university #acco...
freq,1,93,2,6,16


- questions랑 answers 매칭하기
- Idea 1 : questions_title에 있는 명사만 tokenizing으로 뽑아내서 professional_industry와 연결짓기?
- Idea 2 : questions_body에서 NER(개체명인식)을 통해 중요한 개체를 찾아서 TF-idf 돌리기 => 유사한 질문목록 뽑기

### 2-9. School_memberships

In [26]:
school_memberships.head()

,school_memberships_school_id,school_memberships_user_id
0,197406,23dce13ca6164a73aec7a3cd56a4884d
1,197398,23dce13ca6164a73aec7a3cd56a4884d
2,199821,23dce13ca6164a73aec7a3cd56a4884d
3,186239,9c5803ae43ca4cf6b27ea85871625116
4,182063,9c5803ae43ca4cf6b27ea85871625116


In [27]:
school_memberships.describe()

,school_memberships_school_id
count,5638.000000
mean,167256.254346
std,47588.724813
min,69421.000000
25%,125573.500000
50%,196934.000000
75%,198771.000000
max,214607.000000


### 2-10. Students

In [28]:
students.head()

,students_id,students_location,students_date_joined
0,12a89e96755a4dba83ff03e03043d9c0,NaN,2011-12-16 14:19:24 UTC+0000
1,e37a5990fe354c60be5e87376b08d5e3,NaN,2011-12-27 03:02:44 UTC+0000
2,12b402cceeda43dcb6e12ef9f2d221ea,NaN,2012-01-01 05:00:00 UTC+0000
3,a0f431fc79794edcb104f68ce55ab897,NaN,2012-01-01 05:00:00 UTC+0000
4,23aea4702d804bd88d1e9fb28074a1b4,NaN,2012-01-01 05:00:00 UTC+0000


In [29]:
students.describe()

,students_id,students_location,students_date_joined
count,30971,28938,30971
unique,30971,5480,29736
top,1027121e74a84df4ae63113d1c0c84a2,"New York, New York",2013-11-22 05:00:00 UTC+0000
freq,1,1313,246


- 학생들 location도 너무 다양하자나..? 향후에 professional이랑 같은 위치인 사람들끼리 오프라인 매칭 시켜주는 것도 좋을 듯?
- 주로 언제들 조인하는지 2시간 단위로 plot그려보기(새벽시간에 많을 것 같긴 한데)

### 2-11. Tag_questions

In [30]:
tag_questions.head()

,tag_questions_tag_id,tag_questions_question_id
0,28930,cb43ebee01364c68ac61d347a393ae39
1,28930,47f55e85ce944242a5a347ab85a8ffb4
2,28930,ccc30a033a0f4dfdb2eb987012f25792
3,28930,e30b274e48d741f7bf50eb5e7171a3c0
4,28930,3d22742052df4989b311b4195cbb0f1a


In [31]:
tag_questions.describe()

,tag_questions_tag_id
count,76553.000000
mean,17289.690763
std,10308.915329
min,27.000000
25%,11165.000000
50%,18360.000000
75%,26420.000000
max,39250.000000


### 2-12. Tag_users

In [32]:
tag_users.head()

,tag_users_tag_id,tag_users_user_id
0,593,c72ab38e073246e88da7e9a4ec7a4472
1,1642,8db519781ec24f2e8bdc67c2ac53f614
2,638,042d2184ee3e4e548fc3589baaa69caf
3,11093,c660bd0dc1b34224be78a58aa5a84a63
4,21539,8ce1dca4e94240239e4385ed22ef43ce


In [33]:
tag_users.describe()

,tag_users_tag_id
count,136663.000000
mean,18971.528629
std,9018.442845
min,27.000000
25%,18065.000000
50%,18807.000000
75%,24132.000000
max,39261.000000


### 2-13. Tags

In [34]:
tags.head()

,tags_tag_id,tags_tag_name
0,27490,college
1,461,computer-science
2,593,computer-software
3,27292,business
4,18217,doctor


In [35]:
tags.describe()

,tags_tag_id
count,16269.000000
mean,28511.686336
std,8421.414012
min,27.000000
25%,21711.000000
50%,31101.000000
75%,35207.000000
max,39276.000000


- tag 3개 테이블을 한 테이블로 합치기
- Idea: 태그들 top10으로 띄워서 웹페이지에서 해당 태그 누르면 자동으로 해당 top 질문-답변 볼 수 있게 하는 것은 어떨까?

In [64]:
user_n_tag = pd.merge(tags, tag_users, left_on='tags_tag_id', right_on='tag_users_tag_id')
#user_tag = user_tag.drop(['tags_tag_id','tag_users_tag_id'],axis=1)
user_n_tag.sort_values('tag_users_user_id')[:10]

,tags_tag_id,tags_tag_name,tag_users_tag_id,tag_users_user_id
111346,27444,content-creation,27444,00009a0f9bda43eba47104e9ac62aff5
130482,22819,script-writing,22819,00009a0f9bda43eba47104e9ac62aff5
77020,520,digital-media,520,00009a0f9bda43eba47104e9ac62aff5
16036,162,accounting,162,000196ef8db54b9a86ae70ad31745d04
57344,27323,real-estate,27323,0008138be908438e8944b21f7f57f2c1
33607,27510,information-technology,27510,000d4635e5da41e3bfd83677ee11dda4
343,27490,college,27490,000d4635e5da41e3bfd83677ee11dda4
16681,410,university,410,000d4635e5da41e3bfd83677ee11dda4
36283,18324,financial-services,18324,000e2b5714444d79a672bf927905135c
36360,18324,financial-services,18324,0018873fbf7742aba1bf13fff12cbfa4
